In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import samplerate

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
#Initialise dataset arrays
ta = [];
Xa = [];
Ya = [];
Za = [];

#Global Parameters
TrimValue = 0.025;#Defines the minimum accelerometer magnitude to be considered valid
DataLength = 1400;#Defines the overall length off the datasets
DataTrim = 200;#Defines the length of the overall datasets we consider to be dead zones

#Read in all data
for k in range(1,21):
    
    #Read in dataset
    if (k<6):
        cd = pd.read_csv("Forward{num}.csv".format(num=k), header = 0)
    elif (k<11):
        cd = pd.read_csv("Punch{num}.csv".format(num=k-5), header = 0)
    elif(k<16):
        cd = pd.read_csv("SwipeLeft{num}.csv".format(num=k-10), header = 0)
    elif(k<21):
        cd = pd.read_csv("SwipeRight{num}.csv".format(num=k-15), header = 0)
    
    #Extract Required columns
    t = cd.loc[:,"seconds_elapsed"]
    X = cd.loc[:,"accelerationX"]
    Y = cd.loc[:,"accelerationY"]
    Z = cd.loc[:,"accelerationZ"]
    
    #Reshape columns
    t = np.reshape(t,(-1,1))
    X = np.reshape(X,(-1,1))
    Y = np.reshape(Y,(-1,1))
    Z = np.reshape(Z,(-1,1))

    # Trim the data
    flat = 0
    for k in range(len(t)):
        if (np.sqrt(math.pow(X[k], 2) + math.pow(Y[k], 2) + math.pow(Z[k], 2))) < TrimValue:
            flat = flat + 1
    t = t[flat:]
    X = X[flat:]
    Y = Y[flat:]
    Z = Z[flat:]
    flat = 0
    for k in range(len(t)):
        if (np.sqrt(math.pow(X[len(t)-k-1], 2) + math.pow(Y[len(t)-k-1], 2) + math.pow(Z[len(t)-k-1], 2))) < TrimValue:
            flat = flat + 1
    t = t[:len(t)-flat]
    X = X[:len(t)-flat]
    Y = Y[:len(t)-flat]
    Z = Z[:len(t)-flat]
    
    #Normalise the data to be an array of length 1000
    t = samplerate.resample(t,(DataLength+1)/t.shape[0], 'linear')  
    X = samplerate.resample(X,(DataLength+1)/X.shape[0], 'sinc_best')  
    Y = samplerate.resample(Y,(DataLength+1)/Y.shape[0], 'sinc_best')  
    Z = samplerate.resample(Z,(DataLength+1)/Z.shape[0], 'sinc_best')  

    #Trim function to eliminate any possible rounding errors that cause 1001 after resampling
    t = t[0:DataLength]
    X = X[0:DataLength]
    Y = Y[0:DataLength]
    Z = Z[0:DataLength]

    #Trim function to deaden ends
    t1 = DataTrim;
    t2 = DataLength-DataTrim;
    t = t[t1:t2]
    X = X[t1:t2]
    Y = Y[t1:t2]
    Z = Z[t1:t2]
    
    #Normalise the data 
    scalar = MinMaxScaler(feature_range=(0,1))
    X = scalar.fit_transform(X)
    Y = scalar.fit_transform(Y)
    Z = scalar.fit_transform(Z)
    
    #Append columns to data array
    ta.append(np.array(t))
    Xa.append(np.array(X))
    Ya.append(np.array(Y))
    Za.append(np.array(Z))
    
#Convert to Numpy arrays
ta = np.array(ta)
Xa = np.array(Xa)
Ya = np.array(Ya)
Za = np.array(Za)
    
print("ta has dimensions {}".format(ta.shape))
print("Xa has dimensions {}".format(Xa.shape))
print("Ya has dimensions {}".format(Ya.shape))
print("Za has dimensions {}".format(Za.shape))

ta has dimensions (20, 1000, 1)
Xa has dimensions (20, 1000, 1)
Ya has dimensions (20, 1000, 1)
Za has dimensions (20, 1000, 1)


In [3]:
#Creating the CNN model

model = models.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(1000, 3)))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax')) 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\portm\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 998, 64)        │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 996, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 994, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 63616)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       254,468 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,812 (1.07 MB)

 Trainable params: 279,812 (1.07 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
#first 5 recordings represent one gesture and next 5 represent another and so on
labels = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1,2,2,2,2,2,3,3,3,3,3])

fullData = np.stack((Xa, Ya, Za), axis=-1)
fullData = np.squeeze(fullData, axis=2)
#print("Shape of combined data:", fullData.shape)

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(fullData, labels, test_size=0.2)

# Train the model
history = model.fit(train_data, train_labels, epochs=220, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1667 - loss: 1.4122 - val_accuracy: 0.2500 - val_loss: 3.3597
Epoch 2/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6667 - loss: 1.1215 - val_accuracy: 0.0000e+00 - val_loss: 3.8440
Epoch 3/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6667 - loss: 0.8962 - val_accuracy: 0.2500 - val_loss: 3.6978
Epoch 4/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9167 - loss: 0.6345 - val_accuracy: 0.2500 - val_loss: 3.5863
Epoch 5/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8333 - loss: 0.4233 - val_accuracy: 0.2500 - val_loss: 3.5183
Epoch 6/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.2434 - val_accuracy: 0.0000e+00 - val_loss: 3.6942
Epoch 7/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 0.1325 - val_accuracy: 0.0000e+00 - val_loss: 4.1499
Epoch 8/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0694 - val_accuracy: 0.0000e+

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 1.1921e-07 - val_accuracy: 0.2500 - val_loss: 21.1942
Epoch 51/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 1.1921e-07 - val_accuracy: 0.2500 - val_loss: 21.2440
Epoch 52/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 1.1921e-07 - val_accuracy: 0.2500 - val_loss: 21.2894
Epoch 53/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 1.1921e-07 - val_accuracy: 0.2500 - val_loss: 21.3308
Epoch 54/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 1.0928e-07 - val_accuracy: 0.2500 - val_loss: 21.3686
Epoch 55/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 1.0928e-07 - val_accuracy: 0.2500 - val_loss: 21.4031
Epoch 56/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 9.9341e-08 - val_accuracy: 0.2500 - val_loss: 21.4345
Epoch 57/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 9.9341e-08 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7757
Epoch 99/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7774
Epoch 100/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7791
Epoch 101/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7807
Epoch 102/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7823
Epoch 103/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7839
Epoch 104/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 5.9605e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.7855
Epoch 105/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - ac

Epoch 146/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8369
Epoch 147/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8380
Epoch 148/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8392
Epoch 149/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8403
Epoch 150/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8415
Epoch 151/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8426
Epoch 152/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8438
Epoch 153/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

Epoch 194/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8963
Epoch 195/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8976
Epoch 196/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.8990
Epoch 197/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.9003
Epoch 198/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.9016
Epoch 199/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.9030
Epoch 200/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 3.9736e-08 - val_accuracy: 0.0000e+00 - val_loss: 21.9043
Epoch 201/220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [5]:
def prep(name):

    #Global Parameters
    TrimValue = 0.025;#Defines the minimum accelerometer magnitude to be considered valid
    DataLength = 1400;#Defines the overall length off the datasets
    DataTrim = 200;#Defines the length of the overall datasets we consider to be dead zones
    
    #Read in the data
    cd = pd.read_csv(name, header = 0)
    
    #Extract Required columns
    t = cd.loc[:,"seconds_elapsed"]
    X = cd.loc[:,"accelerationX"]
    Y = cd.loc[:,"accelerationY"]
    Z = cd.loc[:,"accelerationZ"]
    
    #Reshape columns
    t = np.reshape(t,(-1,1))
    X = np.reshape(X,(-1,1))
    Y = np.reshape(Y,(-1,1))
    Z = np.reshape(Z,(-1,1))

    # Trim the data
    flat = 0
    for k in range(len(t)):
        if (np.sqrt(math.pow(X[k], 2) + math.pow(Y[k], 2) + math.pow(Z[k], 2))) < TrimValue:
            flat = flat + 1
    t = t[flat:]
    X = X[flat:]
    Y = Y[flat:]
    Z = Z[flat:]
    flat = 0
    for k in range(len(t)):
        if (np.sqrt(math.pow(X[len(t)-k-1], 2) + math.pow(Y[len(t)-k-1], 2) + math.pow(Z[len(t)-k-1], 2))) < TrimValue:
            flat = flat + 1
    t = t[:len(t)-flat]
    X = X[:len(t)-flat]
    Y = Y[:len(t)-flat]
    Z = Z[:len(t)-flat]
    
    #Normalise the data to be an array of length 1000
    t = samplerate.resample(t,(DataLength+1)/t.shape[0], 'linear')  
    X = samplerate.resample(X,(DataLength+1)/X.shape[0], 'sinc_best')  
    Y = samplerate.resample(Y,(DataLength+1)/Y.shape[0], 'sinc_best')  
    Z = samplerate.resample(Z,(DataLength+1)/Z.shape[0], 'sinc_best')  

    #Trim function to eliminate any possible rounding errors that cause 1001 after resampling
    t = t[0:DataLength]
    X = X[0:DataLength]
    Y = Y[0:DataLength]
    Z = Z[0:DataLength]

    #Trim function to deaden ends
    t1 = DataTrim;
    t2 = DataLength-DataTrim;
    t = t[t1:t2]
    X = X[t1:t2]
    Y = Y[t1:t2]
    Z = Z[t1:t2]
    
    #Normalise the data 
    scalar = MinMaxScaler(feature_range=(0,1))
    X = scalar.fit_transform(X)
    Y = scalar.fit_transform(Y)
    Z = scalar.fit_transform(Z)
    
    return np.array([X, Y, Z])

In [6]:
def recognise(gest):
    # Assuming gest is your new gesture data with shape (3, 1000)
    gest = gest.T  # Shape: (1000, 3)

    # Reshape the gesture data to match the input shape expected by the model
    gest = gest.reshape(1, 1000, 3)  # Add batch dimension

    # Classify the gesture using the trained model
    predicted_label = np.argmax(model.predict(gest), axis=-1)

    # Get the predicted class name
    class_names = ['forward','punch','swipeLeft','swipeRight']
    predicted_class_name = class_names[predicted_label.item()]  # Convert predicted_label to scalar value
    
    return predicted_class_name

In [8]:
gest1 = prep("Gesture1.csv") #Gesture1 is a new swipe left gesture
gest2 = prep("Gesture2.csv") #Gesture2 is a new swipe right gesture
gest3 = prep("Gesture3.csv") #Gesture3 is a new punch gesture
gest4 = prep("Gesture4.csv") #Gesture4 is a new forward gesture

predicted_class_name1 = recognise(gest1);
predicted_class_name2 = recognise(gest2);
predicted_class_name3 = recognise(gest3);
predicted_class_name4 = recognise(gest4);


print("Predicted label for gest1:", predicted_class_name1)
print("Predicted label for gest2:", predicted_class_name2)
print("Predicted label for gest3:", predicted_class_name3)
print("Predicted label for gest4:", predicted_class_name4)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted label for gest1: forward
Predicted label for gest2: swipeLeft
Predicted label for gest3: punch
Predicted label for gest4: swipeLeft
